In [22]:
# pip install langchain_community

In [26]:
import os
import numpy as np
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import threading
import glob
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter




In [ ]:
app = Flask(__name__)

# Initialize the transformer pipeline for text generation
qa_pipeline = pipeline("text-generation", model="gpt2")

# Load and index documents
documents = []  # Placeholder for loaded documents
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = None  # Placeholder for TF-IDF matrix



In [4]:
def load_documents(directory_path):

    docs = []
    for file_path in glob.glob(os.path.join(directory_path, '*.txt')):
        with open(file_path, 'r') as file:
            docs.append(file.read())
    return docs

directory_path = '/content/'

docs = load_documents(directory_path)

print(docs)

['Deep Learning is a subset of Machine Learning that uses neural networks with many layers.\nIt has been applied in image recognition, speech processing, and autonomous driving.\n', 'Machine Learning (ML) is a subset of Artificial Intelligence that enables computers to learn from data without being explicitly programmed.\nPopular machine learning algorithms include decision trees, support vector machines, and neural networks.\n', 'Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions.\nAI is being used in various fields such as healthcare, finance, and transportation.\n', 'Natural Language Processing (NLP) is a field of AI that focuses on the interaction between computers and humans using natural language.\nNLP is used in applications like chatbots, sentiment analysis, and machine translation.\n']


In [56]:
def build_index(docs):
    # text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    # docs = text_splitter.split_documents(docs)
    # print(docs)

    tf_model = TfidfVectorizer()
    tf_vector = tf_model.fit_transform(docs)
    # cosine_sim = cosine_similarity(tf_vector)
    return tf_model, tf_vector

tf_model, tf_vector_index = build_index(docs)

In [62]:
def retrieve_top_k(query, tf_model, tf_vector_index, k=2):
    query_vector = tf_model.transform([query])
    # print(query_vector)
    similarity_scores = cosine_similarity(query_vector, tf_vector_index)
    # print(similarity_scores)
    top_indices = np.argsort(similarity_scores[0])[::-1][:k]

    return top_indices

# query = 'healthcare, finance, and transportation'
# top_k_vectors = retrieve_top_k(query, tf_model, tf_vector_index, k=1)
# print(top_k_vectors)

[2]


In [73]:
def get_top_k_documents(query, top_k_vectors, docs, k = 2):
    top_k_vectors = retrieve_top_k(query, tf_model, tf_vector_index, k=k)
    top_k_documents = [docs[i] for i in top_k_vectors]
    return top_k_documents

query = 'What are the application of NLP'
k = 2
retr_doc = get_top_k_documents(query, top_k_vectors, docs, k)

In [79]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


# chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retr_doc)
# chain.invoke(query)

from langchain.prompts import PromptTemplate, chat

# PromptTemplate({
#     "question": query,
#     "context": retr_doc
# })


In [ ]:
pip

In [106]:
# # ************************ One way **********************
# from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
# model_name = "google/flan-t5-base"  # You can choose a different model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# def generate_answer(query, retrieved_docs):
#   context = "\n".join(retrieved_docs)  # Combine retrieved documents into a single string
#   prompt = f"Question: {query}\nContext: {context}"
#   # Use the pipeline to generate an answer
#   # answer = qa_pipeline(prompt)
#   answer = qa_pipeline(query, retrieved_docs)

#   return answer

# query = "What are the application of NLP?"
# retrieved_docs = get_top_k_documents(query, top_k_vectors, docs, k)

# answer = generate_answer(query, retrieved_docs)
# print(f"Answer: {answer}")

# ************************ Another way **********************

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"  # Use a generative model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

def generate_answer(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    prompt = f"Question: {query}\nContext: {context}"

    answer = qa_pipeline(prompt, max_length=100)

    return answer[0]["generated_text"]

query = "What are the application of NLP?"
retrieved_docs = get_top_k_documents(query, top_k_vectors, docs, k)

answer = generate_answer(query, retrieved_docs)
print(f"Answer: {answer}")


Device set to use cpu


Answer: chatbots, sentiment analysis, and machine translation


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

KeyError: "Unknown task question-answer, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [ ]:
@app.route('/rag', methods=['POST'])
def rag_endpoint():
    """
    API endpoint to handle RAG queries.
    :return: JSON response with the generated answer.
    """
    data = request.get_json()
    query = data.get('query')

    if not query:
        return jsonify({"error": "Query is required"}), 400

    # TODO: Retrieve top documents and generate an answer using the GPT-2 pipeline

    pass



In [ ]:
# Run the server in a separate thread
def run_server():
    """
    Run the Flask server in a separate thread.
    """
    app.run(port=6000, debug=False, use_reloader=False)



In [ ]:
if __name__ == '__main__':
    # TODO: Load documents and build the index before starting the server
    thread = threading.Thread(target=run_server)
    thread.start()
